In [1]:
import es_distributed.tf_util as U
import tensorflow as tf

from es_distributed.policies import catcher, CatchPolicy
from es_distributed.es import *
from es_distributed import policies


exp = {
  "config": {
    "calc_obstat_prob": 0.0,
    "episodes_per_batch": 10000,
    "eval_prob": 0.03,
    "l2coeff": 0.005,
    "noise_stdev": 0.02,
    "snapshot_freq": 5,
    "timesteps_per_batch": 100000,
    "return_proc_mode": "centered_rank",
    "episode_cutoff_mode": "env_default"
  },
  "env_id": "catcher",
  "exp_prefix": "humanoid",
  "optimizer": {
    "args": {
      "stepsize": 0.01
    },
    "type": "adam"
  },
  "policy": {
    "args": {
      "connection_type": "ff",
      "hidden_dims": [
        100,
        100
      ],
      "nonlin_type": "tanh"
    },
    "type": "CatchPolicy"
  }
}


config, env, sess, policy = setup(exp, single_threaded=False)
policy.initialize_from('/home/aignatov/snapshot.h5')
policy.rollout(env)

Instructions for updating:
Please use tf.global_variables instead.
Instructions for updating:
Use `tf.variables_initializer` instead.


(array([    0.,     0.,     0.,     0.,     0.,     0.,     0.,  1500.], dtype=float32),
 8)

In [2]:
for i in range(100):
    print(policy.rollout(env)[0][-1])

1500.0
1500.0
1500.0
-1000.0
-1000.0
-1000.0
1000.0
-1000.0
1500.0
1500.0
-1000.0
1500.0
1500.0
1500.0
-1000.0
1500.0
1500.0
1500.0
1500.0
1500.0
1500.0
1500.0
1500.0
1500.0
1500.0
1500.0
1500.0
1000.0
1500.0
1500.0
1500.0
1500.0
1500.0
1000.0
-1000.0
-1000.0
1500.0
-1000.0
1500.0
1500.0
1500.0
1500.0
1500.0
-1000.0
1500.0
1500.0
1500.0
1500.0
1500.0
1500.0
1500.0
1000.0
1500.0
1500.0
1500.0
1500.0
1500.0
1000.0
1500.0
1500.0
1500.0
1500.0
1000.0
1500.0
1500.0
1500.0
1500.0
1500.0
1000.0
1500.0
1500.0
1500.0
1500.0
1500.0
1000.0
1500.0
1500.0
1500.0
1500.0
1500.0
1500.0
1500.0
1000.0
1500.0
1500.0
1000.0
1500.0
1500.0
1500.0
-1000.0
1500.0
1500.0
1500.0
1500.0
1500.0
1000.0
1500.0
1500.0
1500.0
-1000.0


In [3]:
init = tf.initialize_all_variables()
sess.run(init)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [4]:
nextQ = tf.placeholder(shape=policy.a.shape,dtype=tf.float32)
loss = tf.reduce_mean(tf.square(nextQ - policy.a))
#trainer = tf.train.RMSPropOptimizer(0.001, momentum=0.95, epsilon=0.01)
trainer = tf.train.AdamOptimizer(learning_rate=0.01)
updateModel = trainer.minimize(loss)


import json
import numpy as np

class Catch(object):
    def __init__(self, grid_size=10):
        self.grid_size = grid_size
        self.reset()

    def _update_state(self, action):
        """
        Input: action and states
        Ouput: new states and reward
        """
        state = self.state
        if action == 0:  # left
            action = -1
        elif action == 1:  # stay
            action = 0
        else:
            action = 1  # right
        f0, f1, basket = state[0]
        new_basket = min(max(1, basket + action), self.grid_size-1)
        f0 += 1
        out = np.asarray([f0, f1, new_basket])
        out = out[np.newaxis]

        assert len(out.shape) == 2
        self.state = out

    def _draw_state(self):
        im_size = (self.grid_size,)*2
        state = self.state[0]
        canvas = np.zeros(im_size)
        canvas[state[0], state[1]] = 1  # draw fruit
        canvas[-1, state[2]-1:state[2] + 2] = 1  # draw basket
        return canvas

    def _get_reward(self):
        fruit_row, fruit_col, basket = self.state[0]
        if fruit_row == self.grid_size-1:
            if abs(fruit_col - basket) <= 1:
                return 1000
            else:
                return -1000
        else:
            return 0

    def _is_over(self):
        if self.state[0, 0] == self.grid_size-1:
            return True
        else:
            return False

    def observe(self):
        canvas = self._draw_state()
        return canvas.reshape((1, -1))

    def act(self, action):
        self._update_state(action)
        reward = self._get_reward()
        game_over = self._is_over()
        return self.observe(), reward, game_over

    def reset(self):
        n = np.random.randint(0, self.grid_size-1, size=1)
        m = np.random.randint(1, self.grid_size-2, size=1)
        self.state = np.asarray([0, n, m])[np.newaxis]


class ExperienceReplay(object):
    def __init__(self, max_memory=100, discount=.9):
        self.max_memory = max_memory
        self.memory = list()
        self.discount = discount

    def remember(self, states, game_over):
        # memory[i] = [[state_t, action_t, reward_t, state_t+1], game_over?]
        self.memory.append([states, game_over])
        if len(self.memory) > self.max_memory:
            del self.memory[0]

    def get_batch(self, policy, batch_size=10):
        len_memory = len(self.memory)
        num_actions = int(policy.a.shape[-1])
        env_dim = self.memory[0][0][0].shape[1]
        inputs = np.zeros((min(len_memory, batch_size), env_dim))
        targets = np.zeros((inputs.shape[0], num_actions))
        for i, idx in enumerate(np.random.randint(0, len_memory,
                                                  size=inputs.shape[0])):
            state_t, action_t, reward_t, state_tp1 = self.memory[idx][0]
            game_over = self.memory[idx][1]

            inputs[i:i+1] = state_t
            # There should be no target values for actions not taken.
            # Thou shalt not correct actions not taken #deep
            targets[i] = policy.act(state_t)[0]
            Q_sa = np.max(policy.act(state_tp1)[0])
            if game_over:  # if game_over is True
                targets[i, action_t] = reward_t
            else:
                # reward_t + gamma * max_a' Q(s', a')
                targets[i, action_t] = reward_t + self.discount * Q_sa
        return inputs, targets


epsilon = .1  # exploration
num_actions = 3  # [move_left, stay, move_right]
epoch = 100000
max_memory = 500
hidden_size = 100
batch_size = 1
grid_size = 10

#model = Sequential()
#model.add(Dense(hidden_size, input_shape=(grid_size**2,), activation='relu'))
#model.add(Dense(hidden_size, activation='relu'))
#model.add(Dense(num_actions))
#model.compile(sgd(lr=.2), "mse")

# If you want to continue training from a previous model, just uncomment the line bellow
# model.load_weights("model.h5")

# Define environment/game
env = Catch(grid_size)

# Initialize experience replay object
exp_replay = ExperienceReplay(max_memory=max_memory)

# Train
win_cnt = 0
for e in range(epoch):
    loss = 0.
    env.reset()
    game_over = False
    # get initial input
    input_t = env.observe()

    while not game_over:
        input_tm1 = input_t
        # get next action
        if np.random.rand() <= epsilon:
            action = np.random.randint(0, num_actions, size=1)
        else:
            q = policy.act(input_tm1)
            action = np.argmax(q[0])

        # apply action, get rewards and new state
        input_t, reward, game_over = env.act(action)
        if reward > 0:
            win_cnt += 1

        # store experience
        exp_replay.remember([input_tm1, action, reward, input_t], game_over)

    # adapt model
    inputs, targets = exp_replay.get_batch(policy, batch_size=batch_size)

    print(policy.act([catcher().reset()])[0])
    #loss += model.train_on_batch(inputs, targets)[0]
    U.eval(updateModel, {policy.o:inputs,nextQ:targets})
    print(policy.act([catcher().reset()])[0])
    print('\n')

    print("Epoch {:03d}/999 | Win count {}".format(e, win_cnt))



[ 0.0014195  -0.00406937  0.00154286]


FailedPreconditionError: Attempting to use uninitialized value beta2_power
	 [[Node: beta2_power/read = Identity[T=DT_FLOAT, _class=["loc:@CatchPolicy/l0/b"], _device="/job:localhost/replica:0/task:0/cpu:0"](beta2_power)]]

Caused by op 'beta2_power/read', defined at:
  File "/home/aignatov/miniconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/aignatov/miniconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/aignatov/miniconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/aignatov/miniconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/aignatov/miniconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478, in start
    self.io_loop.start()
  File "/home/aignatov/miniconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 17

In [4]:
policy.a.shape

TensorShape([Dimension(None), Dimension(3)])

In [5]:
env = catcher()
for i in range(100):
    print(policy.rollout(env)[0][-1])

-1000.0
1000.0
-1000.0
1500.0
-1000.0
1000.0
1500.0
-1000.0
-1000.0
1000.0
-1000.0
1000.0
-1000.0
-1000.0
-1000.0
-1000.0
1000.0
-1000.0
-1000.0
1000.0
-1000.0
-1000.0
-1000.0
-1000.0
-1000.0
-1000.0
-1000.0
-1000.0
1000.0
-1000.0
-1000.0
-1000.0
-1000.0
-1000.0
1500.0
-1000.0
1000.0
-1000.0
-1000.0
-1000.0
1500.0
1000.0
-1000.0
1000.0
-1000.0
-1000.0
-1000.0
-1000.0
-1000.0
-1000.0
1000.0
-1000.0
1000.0
1000.0
-1000.0
-1000.0
-1000.0
1500.0
1000.0
-1000.0
-1000.0
1500.0
-1000.0
-1000.0
-1000.0
1000.0
-1000.0
-1000.0
-1000.0
-1000.0
1500.0
-1000.0
-1000.0
1000.0
-1000.0
-1000.0
1500.0
-1000.0
1000.0
1000.0
1500.0
-1000.0
-1000.0
-1000.0
1000.0
-1000.0
-1000.0
-1000.0
-1000.0
-1000.0
-1000.0
-1000.0
-1000.0
-1000.0
-1000.0
-1000.0
1000.0
-1000.0
1500.0
-1000.0


# standard Qlearn

In [12]:
import json
import numpy as np
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import sgd


class Catch(object):
    def __init__(self, grid_size=10):
        self.grid_size = grid_size
        self.reset()

    def _update_state(self, action):
        """
        Input: action and states
        Ouput: new states and reward
        """
        state = self.state
        if action == 0:  # left
            action = -1
        elif action == 1:  # stay
            action = 0
        else:
            action = 1  # right
        f0, f1, basket = state[0]
        new_basket = min(max(1, basket + action), self.grid_size-1)
        f0 += 1
        out = np.asarray([f0, f1, new_basket])
        out = out[np.newaxis]

        assert len(out.shape) == 2
        self.state = out

    def _draw_state(self):
        im_size = (self.grid_size,)*2
        state = self.state[0]
        canvas = np.zeros(im_size)
        canvas[state[0], state[1]] = 1  # draw fruit
        canvas[-1, state[2]-1:state[2] + 2] = 1  # draw basket
        return canvas

    def _get_reward(self):
        fruit_row, fruit_col, basket = self.state[0]
        if fruit_row == self.grid_size-1:
            if abs(fruit_col - basket) <= 1:
                return 1000
            else:
                return -1000
        else:
            return 0

    def _is_over(self):
        if self.state[0, 0] == self.grid_size-1:
            return True
        else:
            return False

    def observe(self):
        canvas = self._draw_state()
        return canvas.reshape((1, -1))

    def act(self, action):
        self._update_state(action)
        reward = self._get_reward()
        game_over = self._is_over()
        return self.observe(), reward, game_over

    def reset(self):
        n = np.random.randint(0, self.grid_size-1, size=1)
        m = np.random.randint(1, self.grid_size-2, size=1)
        self.state = np.asarray([0, n, m])[np.newaxis]


class ExperienceReplay(object):
    def __init__(self, max_memory=100, discount=.9):
        self.max_memory = max_memory
        self.memory = list()
        self.discount = discount

    def remember(self, states, game_over):
        # memory[i] = [[state_t, action_t, reward_t, state_t+1], game_over?]
        self.memory.append([states, game_over])
        if len(self.memory) > self.max_memory:
            del self.memory[0]

    def get_batch(self, model, batch_size=10):
        len_memory = len(self.memory)
        num_actions = model.output_shape[-1]
        env_dim = self.memory[0][0][0].shape[1]
        inputs = np.zeros((min(len_memory, batch_size), env_dim))
        targets = np.zeros((inputs.shape[0], num_actions))
        for i, idx in enumerate(np.random.randint(0, len_memory,
                                                  size=inputs.shape[0])):
            state_t, action_t, reward_t, state_tp1 = self.memory[idx][0]
            game_over = self.memory[idx][1]

            inputs[i:i+1] = state_t
            # There should be no target values for actions not taken.
            # Thou shalt not correct actions not taken #deep
            targets[i] = model.predict(state_t)[0]
            Q_sa = np.max(model.predict(state_tp1)[0])
            if game_over:  # if game_over is True
                targets[i, action_t] = reward_t
            else:
                # reward_t + gamma * max_a' Q(s', a')
                targets[i, action_t] = reward_t + self.discount * Q_sa
        return inputs, targets


if __name__ == "__main__":
    # parameters
    epsilon = .1  # exploration
    num_actions = 3  # [move_left, stay, move_right]
    epoch = 50000
    max_memory = 500
    hidden_size = 100
    batch_size = 50
    grid_size = 10

    model = Sequential()
    model.add(Dense(hidden_size, input_shape=(grid_size**2,), activation='relu'))
    model.add(Dense(hidden_size, activation='relu'))
    model.add(Dense(num_actions))
    model.compile(sgd(lr=.2), "mse")

    # If you want to continue training from a previous model, just uncomment the line bellow
    # model.load_weights("model.h5")

    # Define environment/game
    env = Catch(grid_size)

    # Initialize experience replay object
    exp_replay = ExperienceReplay(max_memory=max_memory)

    # Train
    win_cnt = 0
    for e in range(epoch):
        loss = 0.
        env.reset()
        game_over = False
        # get initial input
        input_t = env.observe()

        while not game_over:
            input_tm1 = input_t
            # get next action
            if np.random.rand() <= epsilon:
                action = np.random.randint(0, num_actions, size=1)
            else:
                q = model.predict(input_tm1)
                action = np.argmax(q[0])

            # apply action, get rewards and new state
            input_t, reward, game_over = env.act(action)
            if reward == 1:
                win_cnt += 1
        
        
        # store experience
        exp_replay.remember([input_tm1, action, reward, input_t], game_over)

        # adapt model
        
   
        inputs, targets = exp_replay.get_batch(model, batch_size=batch_size)

        loss += model.train_on_batch(inputs, targets)
        if not e % 20:
            print(model.predict(input_t)[0])
            print("Epoch {:03d}/999 | Loss {:.4f} | Win count {}".format(e, loss, win_cnt))
            print('\n')


[-0.52314901  0.26815662 -0.11703966]
Epoch 000/999 | Loss 0.3931 | Win count 0


[-0.76496881 -0.29357025 -0.45738214]
Epoch 020/999 | Loss 0.0820 | Win count 6


[-0.55678451 -0.78584528  0.01110704]
Epoch 040/999 | Loss 0.1306 | Win count 15


[-0.70658523 -0.4111976  -0.26637673]
Epoch 060/999 | Loss 0.1297 | Win count 21


[-0.61078697 -0.15010278 -0.38725239]
Epoch 080/999 | Loss 0.0669 | Win count 27


[-0.24727343 -1.66710675 -0.84826964]
Epoch 100/999 | Loss 0.0653 | Win count 35


[-0.27554655 -1.07338965 -0.50764942]
Epoch 120/999 | Loss 0.0662 | Win count 41


[-0.5364942  -0.82563227 -0.50051981]
Epoch 140/999 | Loss 0.0668 | Win count 45


[-0.54219073 -0.81450105 -0.37299335]
Epoch 160/999 | Loss 0.0435 | Win count 53


[-0.80052102 -0.3699109  -0.58216071]
Epoch 180/999 | Loss 0.0538 | Win count 60


[-0.66462022 -1.05748928 -0.91957682]
Epoch 200/999 | Loss 0.0371 | Win count 67


[-0.53815812 -0.88478017 -0.65387791]
Epoch 220/999 | Loss 0.0255 | Win count 73


[-0.76

[-0.14288101 -0.29163629 -0.45404333]
Epoch 1960/999 | Loss 0.0005 | Win count 817


[-0.43067124 -1.09726095 -1.08771956]
Epoch 1980/999 | Loss 0.0021 | Win count 824


[ 0.08923449  0.02105755 -0.20762676]
Epoch 2000/999 | Loss 0.0022 | Win count 830


[-0.10822859 -0.25284892 -0.20577809]
Epoch 2020/999 | Loss 0.0075 | Win count 844


[-0.49375916 -0.42131934 -0.10741401]
Epoch 2040/999 | Loss 0.0016 | Win count 853


[ 0.03586473  0.05885044  0.21807949]
Epoch 2060/999 | Loss 0.0067 | Win count 858


[-0.43251523 -1.13688028 -1.05983198]
Epoch 2080/999 | Loss 0.0064 | Win count 866


[-0.21068676 -0.05694053 -0.07424739]
Epoch 2100/999 | Loss 0.0032 | Win count 873


[-0.60829121 -0.83340412 -0.55772769]
Epoch 2120/999 | Loss 0.0039 | Win count 881


[-0.38627851 -0.66381812 -0.68824548]
Epoch 2140/999 | Loss 0.0008 | Win count 888


[-0.44551072  0.05389842  0.03703694]
Epoch 2160/999 | Loss 0.0019 | Win count 898


[ 0.38833565 -0.36508662  0.04524489]
Epoch 2180/999 | Loss 0.000

[-0.28609657 -0.52822584 -0.32029372]
Epoch 3880/999 | Loss 0.0008 | Win count 1552


[-0.15332368 -0.38924983 -0.41888767]
Epoch 3900/999 | Loss 0.0006 | Win count 1561


[ 0.65098625 -0.3413482   0.17474589]
Epoch 3920/999 | Loss 0.0012 | Win count 1572


[-0.95370728 -0.95205092 -0.19278575]
Epoch 3940/999 | Loss 0.0014 | Win count 1579


[-0.65081352 -0.68368655 -0.77859116]
Epoch 3960/999 | Loss 0.0013 | Win count 1585


[ 0.70152736  0.23473878  0.87368244]
Epoch 3980/999 | Loss 0.0015 | Win count 1594


[-0.57012677 -0.51029646 -0.81152433]
Epoch 4000/999 | Loss 0.0233 | Win count 1604


[-0.52650589 -0.03576432 -0.36003491]
Epoch 4020/999 | Loss 0.0005 | Win count 1609


[-0.92457402 -0.62212759 -0.19091761]
Epoch 4040/999 | Loss 0.0041 | Win count 1614


[-0.61142355 -0.52274716 -0.11451997]
Epoch 4060/999 | Loss 0.0022 | Win count 1618


[-0.49152148 -0.52056527 -0.73942935]
Epoch 4080/999 | Loss 0.0007 | Win count 1621


[ 0.01832208  0.06825092  0.32943821]
Epoch 4100/999 |

[ 0.04305717 -0.08667633 -0.0689635 ]
Epoch 5800/999 | Loss 0.0005 | Win count 2290


[-0.45669466 -0.75853395 -0.57592499]
Epoch 5820/999 | Loss 0.0001 | Win count 2297


[-0.26857543 -0.66001326 -0.446747  ]
Epoch 5840/999 | Loss 0.0004 | Win count 2305


[ 0.01829518 -0.14205329  0.12431978]
Epoch 5860/999 | Loss 0.0005 | Win count 2313


[-0.15182362 -0.35019845 -0.43173003]
Epoch 5880/999 | Loss 0.0001 | Win count 2322


[ 0.00880757 -0.15338011 -0.27448022]
Epoch 5900/999 | Loss 0.0001 | Win count 2333


[-0.2883856  -0.19969983 -0.64050108]
Epoch 5920/999 | Loss 0.0013 | Win count 2339


[-0.31668037 -0.66904932 -0.39588097]
Epoch 5940/999 | Loss 0.0007 | Win count 2347


[-0.2609756  -0.7307781  -0.53677952]
Epoch 5960/999 | Loss 0.0003 | Win count 2357


[-0.44742504 -0.28048274 -0.65242553]
Epoch 5980/999 | Loss 0.0011 | Win count 2363


[ 0.69151533 -0.55463809 -0.0327882 ]
Epoch 6000/999 | Loss 0.0008 | Win count 2373


[-0.30124485 -0.70142043 -0.4320758 ]
Epoch 6020/999 |

[-0.19916669 -0.22187331 -0.14875278]
Epoch 7720/999 | Loss 0.0048 | Win count 3128


[-0.0735445  -0.17045543 -0.30985498]
Epoch 7740/999 | Loss 0.0029 | Win count 3135


[-0.74919128 -0.25581583 -0.37056443]
Epoch 7760/999 | Loss 0.0009 | Win count 3142


[-0.08417642 -0.14658906 -0.52696854]
Epoch 7780/999 | Loss 0.0006 | Win count 3150


[ 0.66297531 -0.34041578 -0.32316485]
Epoch 7800/999 | Loss 0.0001 | Win count 3160


[-0.02165966 -0.40152413 -0.98865712]
Epoch 7820/999 | Loss 0.0054 | Win count 3171


[-0.01201412 -0.04095753 -0.20733871]
Epoch 7840/999 | Loss 0.0001 | Win count 3177


[ 0.06276543 -0.62166601 -0.64452654]
Epoch 7860/999 | Loss 0.0001 | Win count 3187


[ 0.06673294 -0.42963561 -0.37760124]
Epoch 7880/999 | Loss 0.0002 | Win count 3194


[-0.32722509 -0.23688811 -0.16213857]
Epoch 7900/999 | Loss 0.0002 | Win count 3202


[-0.31845099 -0.753227   -0.19569124]
Epoch 7920/999 | Loss 0.0001 | Win count 3206


[  9.59664583e-04  -7.88950980e-01  -9.96344030e-01]
E

...

[-0.19711955 -0.56241179 -0.19400778]
Epoch 41720/999 | Loss 0.0000 | Win count 17097


[ 0.15151174 -1.02629495 -0.86609888]
Epoch 41740/999 | Loss 0.0000 | Win count 17106


[-0.37808681 -0.15299843 -0.64312619]
Epoch 41760/999 | Loss 0.0000 | Win count 17111


[-0.29621562 -0.26843911 -0.42821309]
Epoch 41780/999 | Loss 0.0000 | Win count 17119


[-0.41408354 -0.90361011 -0.12120152]
Epoch 41800/999 | Loss 0.0000 | Win count 17127


[-0.36774054 -0.35814524  0.02476309]
Epoch 41820/999 | Loss 0.0000 | Win count 17136


[-0.79959023 -0.38048178 -0.56073582]
Epoch 41840/999 | Loss 0.0000 | Win count 17144


[ 0.29470327 -0.16207451  0.02719171]
Epoch 41860/999 | Loss 0.0000 | Win count 17154


[-0.2015273  -0.55446148 -0.19505155]
Epoch 41880/999 | Loss 0.0000 | Win count 17161


[-0.21654037 -0.2995691  -0.5605011 ]
Epoch 41900/999 | Loss 0.0000 | Win count 17171


[ 0.78047436 -0.15455505 -0.24498467]
Epoch 41920/999 | Loss 0.0000 | Win count 17177


[ 0.12205113 -0.37279043 -0.0677

[ 0.12473498 -0.32253939 -0.08796746]
Epoch 43600/999 | Loss 0.0000 | Win count 17898


[ 0.12369414 -0.32286012 -0.08883239]
Epoch 43620/999 | Loss 0.0000 | Win count 17906


[-0.11106365 -0.35295749 -0.27015138]
Epoch 43640/999 | Loss 0.0000 | Win count 17913


[-0.04833394 -0.18622109 -0.29844549]
Epoch 43660/999 | Loss 0.0000 | Win count 17920


[ 0.30122933 -0.29394984 -0.56370103]
Epoch 43680/999 | Loss 0.0000 | Win count 17927


[-0.76568425 -0.44938388 -0.37418941]
Epoch 43700/999 | Loss 0.0000 | Win count 17932


[-0.22319534 -0.33877185 -0.55431843]
Epoch 43720/999 | Loss 0.0000 | Win count 17942


[ 0.39885762 -0.17847773 -0.03947336]
Epoch 43740/999 | Loss 0.0000 | Win count 17948


[-0.33281007 -0.35670972 -0.43748209]
Epoch 43760/999 | Loss 0.0000 | Win count 17957


[-0.09949536 -0.39311227 -0.30743679]
Epoch 43780/999 | Loss 0.0000 | Win count 17966


[ 0.20673761 -0.20294517 -0.40863672]
Epoch 43800/999 | Loss 0.0000 | Win count 17973


[-0.36098886 -0.167404   -0.6545

[-0.39958262 -0.89447916 -0.14062694]
Epoch 45480/999 | Loss 0.0001 | Win count 18711


[-0.40840062 -0.89888674 -0.1399143 ]
Epoch 45500/999 | Loss 0.0000 | Win count 18720


[-0.40983999 -0.90909266 -0.14276457]
Epoch 45520/999 | Loss 0.0000 | Win count 18728


[-0.04784976 -0.16902345 -0.30195469]
Epoch 45540/999 | Loss 0.0000 | Win count 18736


[-0.06708381 -0.31766319 -0.2525039 ]
Epoch 45560/999 | Loss 0.0000 | Win count 18748


[-0.20259501 -0.3707526  -0.58240432]
Epoch 45580/999 | Loss 0.0000 | Win count 18759


[-0.2797583  -0.59231758 -1.11340928]
Epoch 45600/999 | Loss 0.0000 | Win count 18767


[-0.30377683 -0.1311536   0.03415487]
Epoch 45620/999 | Loss 0.0000 | Win count 18777


[ 0.015526   -0.34392288 -0.42287084]
Epoch 45640/999 | Loss 0.0001 | Win count 18786


[ 0.33042514 -0.3011193  -0.00149043]
Epoch 45660/999 | Loss 0.0000 | Win count 18794


[ 0.21817416 -0.21973726 -0.41741839]
Epoch 45680/999 | Loss 0.0000 | Win count 18801


[-0.20117979 -0.36760554 -0.5803

[-0.19366282 -0.28006613 -0.61438262]
Epoch 47360/999 | Loss 0.0000 | Win count 19573


[ 0.79440033 -0.12757021 -0.38022095]
Epoch 47380/999 | Loss 0.0000 | Win count 19582


[-0.1113753  -0.42679247 -0.41324455]
Epoch 47400/999 | Loss 0.0000 | Win count 19593


[-0.37331152 -0.51699525 -0.06074867]
Epoch 47420/999 | Loss 0.0000 | Win count 19597


[-0.00345539 -0.33915713 -0.42635998]
Epoch 47440/999 | Loss 0.0000 | Win count 19607


[-0.11074962 -0.41177428 -0.40934017]
Epoch 47460/999 | Loss 0.0000 | Win count 19617


[-0.28369117 -0.52881789 -1.21184146]
Epoch 47480/999 | Loss 0.0000 | Win count 19628


[-0.5171206  -0.68880481 -0.71699739]
Epoch 47500/999 | Loss 0.0000 | Win count 19637


[-0.28438962 -0.24636312 -0.14191383]
Epoch 47520/999 | Loss 0.0000 | Win count 19644


[-0.37937319 -0.81567425 -0.64769411]
Epoch 47540/999 | Loss 0.0000 | Win count 19651


[-0.37323812 -0.23717389  0.09185698]
Epoch 47560/999 | Loss 0.0000 | Win count 19660


[ 0.2155174  -0.22852236 -0.4076

[ 0.0078352  -0.34142157 -0.42455658]
Epoch 49240/999 | Loss 0.0000 | Win count 20387


[-0.05104415 -0.18567502 -0.30658677]
Epoch 49260/999 | Loss 0.0000 | Win count 20397


[-0.03582812 -0.31492889 -0.23688312]
Epoch 49280/999 | Loss 0.0000 | Win count 20405


[-0.27212963 -0.11925817  0.06586813]
Epoch 49300/999 | Loss 0.0000 | Win count 20415


[-0.31770885 -0.34411886 -0.73625422]
Epoch 49320/999 | Loss 0.0000 | Win count 20419


[-0.18287355 -0.23271096 -0.56799078]
Epoch 49340/999 | Loss 0.0000 | Win count 20426


[-0.18214443 -0.24253559 -0.57115948]
Epoch 49360/999 | Loss 0.0000 | Win count 20433


[ 0.08773279 -0.26165444 -0.05505484]
Epoch 49380/999 | Loss 0.0000 | Win count 20439


[-0.17962582 -0.2427592  -0.56748229]
Epoch 49400/999 | Loss 0.0000 | Win count 20449


[ 0.29378247 -0.20895261 -0.19998908]
Epoch 49420/999 | Loss 0.0000 | Win count 20455


[-0.1650613  -0.50013661 -0.2324394 ]
Epoch 49440/999 | Loss 0.0000 | Win count 20463


[ 0.20451325 -0.22823629 -0.3934

In [6]:
env = catcher()
policy.act([env.reset()])[0]

array([ 615.98175049,  636.49072266,  616.65515137], dtype=float32)

In [13]:
loss

1.2342154320776899e-07

NameError: name 'inputs' is not defined

In [ ]:
_,W1 = sess.run([updateModel,W],feed_dict={inputs1:inputs,nextQ:targets})